In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
root = '../working/offline/6/0'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1'
# pairwise_model_name = 'deberta-v3-small.flag-pairwise9'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-2'

In [3]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [4]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [5]:
ids = set(xc['id'])

In [6]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [7]:
calc_metric(df_gt, xc, 'reg_pred')

0.8894696501312026

In [8]:
calc_metric(df_gt, xp, 'cls_pred')

0.8809291684348116

In [9]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(df_gt, x)

0.8920296181457701

In [10]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [14]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['pred'], xp['max_prob'])]
calc_metric(df_gt, x)

0.8987078419974337

In [13]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['reg_pred'], xp['max_prob'])]
calc_metric(df_gt, x)

0.8979546928205726

In [15]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['cls_pred'], xp['max_prob'])]
calc_metric(df_gt, x)

0.8986262439922853

In [16]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['cls_pred'], xc['cls_pred'], xp['max_prob'])]
calc_metric(df_gt, x)

0.8903586043422993